In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist
 

from tensorflow.keras.preprocessing import image

from keras.applications.imagenet_utils import decode_predictions

from tensorflow.keras.applications.imagenet_utils import preprocess_input

from tensorflow.keras.applications.resnet50 import preprocess_input

import os

from keras.applications.resnet import ResNet50

from keras.applications.vgg16 import VGG16

from keras.applications.inception_v3 import InceptionV3

In [2]:
# Function for plotting model loss
def plot_loss(history):
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']

    # Set figure size
    plt.figure(figsize=(12, 8))

    # Generate line plot of training, testing loss over epochs
    plt.plot(train_loss, label='Training Loss', color='#185fad')
    plt.plot(test_loss, label='Testing Loss', color='orange')

    # Set title
    plt.title('Training and Testing Loss by Epoch', fontsize = 25)
    plt.xlabel('Epoch', fontsize = 18)
    plt.ylabel('Categorical Crossentropy', fontsize = 18)
    plt.xticks(np.arange(10), np.arange(10))

    plt.legend(fontsize = 18);

In [3]:
model = ResNet50()

2022-10-13 14:17:53.887539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
print(model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [5]:
data_path = './training'
data_dir = os.listdir(data_path)

In [6]:
data_dir # this is the data folder

['drinks',
 'ads',
 'vegetarian',
 'diserts',
 '.DS_Store',
 'places',
 'clutter',
 'non_vegetarian']

In [7]:
data_dir.remove('.DS_Store')

In [8]:
vegetarian_images = os.listdir(data_path+'/vegetarian')
len(vegetarian_images)

447

In [9]:
non_vegetarian_images = os.listdir(data_path+'/non_vegetarian')
len(non_vegetarian_images)

994

In [10]:
places_images = os.listdir(data_path+'/places')
len(places_images)

1230

In [11]:
img_data_list = []
labels = []

for data in data_dir:
    img_list = os.listdir(data_path+'/'+data)
    print('Loading images of', data)
    for img in img_list:
        img_path = data_path +'/'+ data + '/' + img
        try:
            img = image.load_img(img_path, target_size= (224,224))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            img_data_list.append(x)
            labels.append(data)
        except:
            continue

Loading images of drinks
Loading images of ads


/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Loading images of vegetarian
Loading images of diserts
Loading images of places
Loading images of clutter
Loading images of non_vegetarian


In [12]:
img_data_list[0]

array([[[ -87.939    , -108.779    ,  -32.68     ],
        [ -87.939    , -107.779    ,  -33.68     ],
        [ -86.939    , -108.779    ,  -35.68     ],
        ...,
        [ -81.939    , -106.779    , -115.68     ],
        [ -83.939    , -108.779    , -115.68     ],
        [ -83.939    , -108.779    , -115.68     ]],

       [[ -87.939    , -108.779    ,  -32.68     ],
        [ -87.939    , -107.779    ,  -33.68     ],
        [ -86.939    , -108.779    ,  -35.68     ],
        ...,
        [ -80.939    , -105.779    , -114.68     ],
        [ -83.939    , -108.779    , -115.68     ],
        [ -83.939    , -108.779    , -115.68     ]],

       [[ -87.939    , -108.779    ,  -32.68     ],
        [ -87.939    , -107.779    ,  -33.68     ],
        [ -86.939    , -108.779    ,  -35.68     ],
        ...,
        [ -80.939    , -105.779    , -114.68     ],
        [ -83.939    , -108.779    , -115.68     ],
        [ -83.939    , -108.779    , -115.68     ]],

       ...,

      

In [13]:
img_data = np.array(img_data_list)

In [14]:
img_data.shape

(3147, 224, 224, 3)

In [15]:
np.save('train_data.npy', img_data)

In [16]:
np.save('train_labels.npy', labels)

In [17]:
img_data = np.load('train_data.npy')
labels = np.load('train_labels.npy')

In [18]:
len(img_data)

3147

In [19]:
len(labels)

3147

In [20]:
labels

array(['drinks', 'drinks', 'drinks', ..., 'non_vegetarian',
       'non_vegetarian', 'non_vegetarian'], dtype='<U14')

In [21]:
num_class = 7
num_of_samples = img_data.shape[0]

In [22]:
num_of_samples

3147

In [23]:
from tensorflow.keras.utils import to_categorical

In [24]:
names = ['vegetarian', 'places', 'non_vegetarian']

In [25]:
from tensorflow.keras import utils

In [26]:
num_labels = []

In [27]:
for each in labels:
    if each == 'vegetarian':
        num_labels.append(0)
    elif each == 'non_vegetarian':
        num_labels.append(1)
    elif each == 'places':
        num_labels.append(2)
    elif each == 'diserts':
        num_labels.append(3)
    elif each == 'drinks':
        num_labels.append(4)
    elif each == 'clutter':
        num_labels.append(5)
    elif each == 'ads':
        num_labels.append(6)

In [28]:
num_labels= np.array(num_labels)

In [29]:
num_labels

array([4, 4, 4, ..., 1, 1, 1])

In [30]:
num_labels[:2]

array([4, 4])

In [31]:
labels

array(['drinks', 'drinks', 'drinks', ..., 'non_vegetarian',
       'non_vegetarian', 'non_vegetarian'], dtype='<U14')

In [32]:
x = img_data

In [33]:
y = utils.to_categorical(num_labels, num_classes=7)

In [34]:
y[:2]

array([[0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

# Train Test Split:

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42, shuffle=True)

In [37]:
X_train.shape

(2517, 224, 224, 3)

In [38]:
X_test.shape

(630, 224, 224, 3)

# Modeling :

In [47]:
from keras.layers import Input, Flatten, Dense

In [48]:
from keras.models import Model

In [49]:
from tensorflow.keras.optimizers import Adam

In [50]:
image_input = Input(shape=(224, 224,3))

In [51]:
model1 = ResNet50(input_tensor= image_input, include_top=True, weights= 'imagenet')
model1.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [52]:
last_layer = model1.get_layer('avg_pool').output

In [53]:
last_layer

<KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'avg_pool')>

In [54]:
x = Flatten(name='Flatten')(last_layer)

In [55]:
out = Dense(7, activation='softmax', name = 'output_layer')(x)

In [56]:
custom_resnet_model = Model(image_input, out)

In [57]:
custom_resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [58]:
for layer in custom_resnet_model.layers[:-1]:
    layer.trainable=False
    
custom_resnet_model.layers[-1].trainable

True

In [59]:
custom_resnet_model.compile(loss='categorical_crossentropy',
                            optimizer= Adam(learning_rate=0.0001),
                            metrics=['acc'])

# Fitting the Model:

In [60]:
X_train.shape

(2517, 224, 224, 3)

In [61]:
history = custom_resnet_model.fit(X_train, y_train, 
                        batch_size=32,
                        epochs=15,
                        verbose=1,
                        validation_data=(X_test, y_test))

Epoch 1/15


ValueError: in user code:

    File "/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 894, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 987, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/opt/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 480, in update_state
        self.build(y_pred,

In [ ]:
plot_loss(history)

In [ ]:
from tensorflow.keras.models import load_model

custom_resnet_model.save('resnet3.h5')

new_model = load_model('resnet3.h5')

new_model.summary()

# Predicting on validation data:

In [74]:
data_path = './testing'
data_dir = os.listdir(data_path)

In [75]:
data_dir.remove('.DS_Store')

In [76]:
data_dir

['drinks',
 'ads',
 'vegetarian',
 'diserts',
 'places',
 'clutter',
 'non_vegetarian']

In [77]:
img_data_list = []
labels = []

for data in data_dir:
    img_list = os.listdir(data_path+'/'+data)
    print('Loading images of', data)
    for img in img_list:
        img_path = data_path+'/'+data + '/' +img
        try:
            img = image.load_img(img_path, target_size= (224,224))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            img_data_list.append(x)
            labels.append(data)
        except:
            continue

Loading images of drinks
Loading images of ads
Loading images of vegetarian
Loading images of diserts
Loading images of places
Loading images of clutter
Loading images of non_vegetarian


In [78]:
img_data = np.array(img_data_list)

In [79]:
img_data.shape

(209, 224, 224, 3)

In [80]:
np.save('testing_data.npy', img_data)

In [81]:
np.save('testing_labels.npy', labels)

In [82]:
img_data = np.load('testing_data.npy')
labels = np.load('testing_labels.npy')

In [83]:
len(img_data)

209

In [84]:
len(labels)

209

In [85]:
labels

array(['drinks', 'drinks', 'drinks', 'drinks', 'drinks', 'drinks',
       'drinks', 'drinks', 'drinks', 'drinks', 'drinks', 'drinks',
       'drinks', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads',
       'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetari

In [86]:
num_class = 3
num_of_samples = img_data.shape[0]

In [87]:
num_of_samples

209

In [88]:
from tensorflow.keras.utils import to_categorical

In [89]:
names = ['vegetarian', 'places', 'non_vegetarian']

In [90]:
from tensorflow.keras import utils

In [91]:
num_labels = []

In [92]:
for each in labels:
    if each == 'vegetarian':
        num_labels.append(0)
    elif each == 'non_vegetarian':
        num_labels.append(1)
    else:
        num_labels.append(2)

In [93]:
num_labels= np.array(num_labels)

In [94]:
num_labels

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [95]:
num_labels[:2]

array([2, 2])

In [96]:
labels

array(['drinks', 'drinks', 'drinks', 'drinks', 'drinks', 'drinks',
       'drinks', 'drinks', 'drinks', 'drinks', 'drinks', 'drinks',
       'drinks', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads',
       'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads', 'ads',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetarian', 'vegetarian', 'vegetarian',
       'vegetarian', 'vegetari

In [97]:
x = img_data

In [98]:
y = utils.to_categorical(num_labels, num_classes=3)

In [99]:
custom_resnet_model.evaluate(x,y, batch_size=10, verbose=1)

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/anaconda3/lib/python3.9/asyncio/base_events.py", line 596, in r

In [ ]:
plt.imshow(X_test[156])

In [101]:
from tensorflow.keras.models import load_model

custom_resnet_model.save('resnet2.h5')

new_model = load_model('resnet2.h5')

new_model.summary()